# Strava segment data

Choose some activity ID or the LAST one if `LAST=True`

In [1]:
activity=3158056389
LAST=True

In [2]:
import helium
import pandas as pd
import os
import time
def metadata(**kwargs):
    return kwargs
db_dir='db'
db_file='{}/strava.json'.format(db_dir)

In [3]:
os.makedirs(db_dir,exist_ok=True)

if os.path.isfile(db_file):
    df=pd.read_json(db_file)
else:
    df=pd.DataFrame()

browser=helium.start_chrome("https://www.strava.com/login")

helium.write('diego.restrepo@gmail.com',into='Your Email')

helium.write('opoqdh11',into='Password')

helium.click('Log In')

time.sleep(10)
browser.maximize_window()

if LAST:
    #Required to activate card-section below
    lt=[t for t in browser.find_elements_by_class_name('card-section') 
       if t.text.find('Latest Activity')>-1][0]

    lt.find_element_by_tag_name('a').click()

else:
    browser.get('https://www.strava.com/activities/{}'.format(activity))
# Web page with latest activity

# General info

time.sleep(10)
dt=browser.find_element_by_class_name('details')

date=dt.find_element_by_tag_name('time').text

name=dt.find_element_by_tag_name('h1').text

kk=browser.find_elements_by_tag_name("ul")

q=[ k for k in kk if k.text.find('\nDistance')>-1][0]

distance,time,elevation=(t.text for t in q.find_elements_by_tag_name('strong') )

q=[ k for k in kk if k.text.find('\nEstimated Avg Power')>-1][0]

power,energy=(t.text for t in q.find_elements_by_tag_name('strong') )

# Load tables

HIDDEN=True
try:
    browser.find_element_by_id('show-hidden-efforts').click()
except:
    HIDDEN=False
    pass

html=browser.page_source

ll=pd.read_html(html)

# General info table

mtdt=ll[0].groupby('Unnamed: 0').sum().transpose().to_dict()

mtdt.update(  metadata(date=date,name=name,distance=distance,time=time,
         elevation=elevation,power=elevation,energy=energy)
   )

#Fix time
import time

# Segments

sgmt=ll[1][['Name','Time','Speed','Power']].reset_index(drop=True)

if HIDDEN:
    sgmt=sgmt.append( ll[2].rename({2:'Name',4:'Time',5:'Speed',6:'Power'},
            axis='columns')[['Name','Time','Speed','Power']] 
           ).reset_index(drop=True)

#Add general info
sgmt['Info']=''

sgmt['Info'] = [ mtdt  for i in sgmt.index]

In [4]:
sgmt

,Name,Time,Speed,Power,Info
0,Subida desde la glorieta sur america 1.34km ...,9:08,8.9km/h,206W,"{'Elapsed Time': {'Avg': '53:13', 'Max': '53:1..."
1,Loma Mall Suramerica 0.31km 17m 4%,1:43,11.0km/h,175W,"{'Elapsed Time': {'Avg': '53:13', 'Max': '53:1..."
2,Variante a Sabaneta 1.91km 127m -4%,7:24,15.5km/h,141W,"{'Elapsed Time': {'Avg': '53:13', 'Max': '53:1..."
3,LOMA LINDA SABANETA 0.46km 82m 18%,6:51,4.1km/h,249W,"{'Elapsed Time': {'Avg': '53:13', 'Max': '53:1..."
4,Glorieta SurA Via vieja la estrella 1.30km 6...,9:01,8.7km/h,206W,"{'Elapsed Time': {'Avg': '53:13', 'Max': '53:1..."
5,Colegio Londres - Bajando Pan de Azucar 1.38k...,5:23,15.4km/h,151W,"{'Elapsed Time': {'Avg': '53:13', 'Max': '53:1..."
6,"Loma linda, detrás del antiguo principito 0.4...",6:55,4.0km/h,247W,"{'Elapsed Time': {'Avg': '53:13', 'Max': '53:1..."


In [5]:
df=df.append(sgmt,sort=False).reset_index(drop=True)

In [6]:
df.to_json(db_file)

### Save notebook here

In [10]:
f=open('commit.sh','w')
f.write('git commit -am "{}"\n'.format(
    df.iloc[-1]['Info']['name']+' '+df.iloc[-1]['Info']['date']
))
f.write('git push origin master')
f.close()

In [8]:
!bash commit.sh

[master 444f8db] Morning Ride 9:18 AM on Sunday, June 21, 2020
 2 files changed, 67 insertions(+), 27 deletions(-)
 rewrite db/strava.json (99%)
Counting objects: 5, done.
Delta compression using up to 8 threads.
Compressing objects: 100% (4/4), done.
Writing objects: 100% (5/5), 4.98 KiB | 0 bytes/s, done.
Total 5 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To github.com:restrepo/strava.git
   136a4f6..444f8db  master -> master


In [9]:
if not( input('Close browser?').find('n')>-1 ):
    browser.close()

Close browser? 
